In [272]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import sys
import collections
import time
from math import gamma
import cPickle as pickle

In [273]:
def getSubstring(substring,string):
    matchs = []
    for ii in range(0,len(string)-len(substring)):
        if string[ii:ii+len(substring)] == substring:
            matchs.append(ii)
    if substring == '"tweet"':
        assert len(matchs) > 0, "no text"
        start = matchs[0]+8
        while string[start:start+10] != '"hashtags"':
            start = start + 1
        return string[matchs[0]+8:start-1].lower()
    elif substring == '"hashtags"':
        for ii in matchs:
            start = ii+11
            end = ii+11
            while string[end:end+3] != '}\n':
                end = end+1
            hashtags = string[start:end]
            if hashtags != '""':
                hashtags = np.copy(hashtags.split(','))
                hashtags = np.copy(hashtags[:-1])
                hashtags[0] = hashtags[0][1:]
            else:
                hashtags = ""
        return np.array([ii.lower() for ii in hashtags])
    elif substring == '"timestamp"':
        start = matchs[0]+12
        while string[start:start+11] != '"screename"':
            start = start + 1
        return string[matchs[0]+12:start-1]
    elif substring == '"screename"':
        start = matchs[0]+12
        while string[start:start+10] != '"location"':
            start = start + 1
        return string[matchs[0]+12:start-1]
    elif substring == '"location"':
        start = matchs[0]+11
        while string[start:start+5] != '"geo"':
            start = start + 1
        return string[matchs[0]+11:start-1]
    else: # '"geo"'
        start = matchs[0]+6
        while string[start:start+7] != '"tweet"':
            start = start + 1
        return string[matchs[0]+6:start-1]

In [278]:
def preProcessWords(texts):
    wordDocDict = {}
    wordDocDictNU = {}
    nDocs = len(texts)
    for ii in xrange(0,nDocs):
        textString = texts[ii].split()
        for jj in xrange(0,len(textString)):
            if jj == 0:
                tempText = textString[jj][1:]
            elif jj == len(textString)-1:
                tempText = textString[jj][:-1]
            else:
                tempText = textString[jj]
            if tempText in wordDocDict:
                wordDocDict[tempText].add(ii)
                wordDocDictNU[tempText].append(ii)
            else:
                wordDocDict[tempText] = set([ii])
                wordDocDictNU[tempText] = [ii]
    uniqueWords = np.array(wordDocDict.keys())
    uniqueWordDocLabels = np.array(wordDocDict.values())
    nuWordDocLabels = np.array(wordDocDictNU.values())
    nDocPerWord = np.array(map(lambda x: len(x),uniqueWordDocLabels))
    
    # only keep words that appear in at least 5 tweets
    uniqueWords = uniqueWords[nDocPerWord > 4]
    nuWordDocLabels = nuWordDocLabels[nDocPerWord > 4]
    nInstances = np.array(map(lambda x: len(x),nuWordDocLabels))
    
    allWords = []
    docLabel = []
    
    remW = set([ii.lower() for ii in ['@', 'me', 'my', '-', 'the', 'is', 'it', 'in', 'just',\
     'for', 'was', 'no', 'when', 'not', 'that', 'and', 'take',\
     'get',  'I', 'on', 'of', 'with', 'at', 'you', 'all', 'to',\
     "I'm", 'a', "don't",'The', 'are', 'back', 'be', 'up', 'go',\
     'from', 'about', 'this', 'do', 'out', 'have',\
     'so', 'will', 'like', '&amp;', 'w\\/', 'but']])
    
    filter1 = np.array([True]*len(uniqueWords))
    for ii in xrange(0,len(uniqueWords)):
        if uniqueWords[ii] in remW:
            filter1[ii] = False
    
    uniqueWords = uniqueWords[filter1]
    nuWordDocLabels = nuWordDocLabels[filter1]
    nInstances = nInstances[filter1]
    
    for ii in xrange(0,len(uniqueWords)):
        allWords = allWords + [uniqueWords[ii]]*len(nuWordDocLabels[ii])
        docLabel = docLabel + nuWordDocLabels[ii]
        
    return np.array(allWords), np.array(docLabel), uniqueWords, nInstances

In [269]:
data = open('zipTweets_20.txt','r')
tweets = data.read()
data.close()

tweets = tweets.split('&&&&&')[:-1]

textsTest = [tweet[:-1] for tweet in tweets]

In [280]:
allWords31,docLabel31,u31,n31 = preProcessWords(textsTest)

In [295]:
print len(allWords31)
print len(np.unique(allWords31))
print len(u31)
print len(np.unique(docLabel31))
print len(textsTest)

17498
957
957
3052
3076


In [282]:
def drawTopicInit(allWords,docLabel,nTopics,alpha,beta):
    
    vocab = np.unique(allWords)
    nVocab = len(vocab)
    wordTopicCounters = {ii:collections.Counter() for ii in vocab} 
    docTopicCounters = {ii:collections.Counter() for ii in np.unique(docLabel)}
    topicCounters = {ii:0 for ii in xrange(0,nTopics)}
    docCounters = {ii:0 for ii in np.unique(docLabel)}
    
    topicVector = np.zeros(len(allWords))
    
    for ii in xrange(0,len(allWords)):
        currentWord = allWords[ii]
        currentDoc = docLabel[ii]
        
        probVector = np.zeros(nTopics)
        for jj in xrange(0,nTopics):
            probVector[jj] = (wordTopicCounters[currentWord][jj]+beta)/(topicCounters[jj] + nVocab*beta)
            probVector[jj] = probVector[jj]*(docTopicCounters[currentDoc][jj]+alpha)/(docCounters[currentDoc] + nTopics*alpha)
        
        probVector = probVector/sum(probVector)
        probCumSum = np.cumsum(probVector)
        randInt = np.random.uniform(0,1,1)

        currentTopic = sum(probCumSum < randInt)
        topicVector[ii] = currentTopic
        
        wordTopicCounters[currentWord][currentTopic] += 1
        docTopicCounters[currentDoc][currentTopic] += 1
        topicCounters[currentTopic] += 1
        docCounters[currentDoc] += 1  
    
    return topicVector, wordTopicCounters, docTopicCounters, topicCounters, docCounters

In [284]:
def drawTopic(wordIndex,allWords,topicLabel,docLabel,nTopics,wordTopicCounters, docTopicCounters, \
                                                                      topicCounters, docCounters,alpha,beta):
    nVocab = len(wordTopicCounters.keys())
    probVector = np.zeros(nTopics)
    
    currentWord = allWords[wordIndex]
    currentDoc = docLabel[wordIndex]
    currentTopic = topicLabel[wordIndex]
    
    for ii in xrange(0,nTopics):
        if ii == currentTopic:
            probVector[ii] = (wordTopicCounters[currentWord][ii]+beta-1)/(topicCounters[ii] + nVocab*beta-1)
            probVector[ii] = probVector[ii]*(docTopicCounters[currentDoc][ii]+alpha-1)/(docCounters[currentDoc] + \
                                                                                        nTopics*alpha - 1)
        else:
            probVector[ii] = (wordTopicCounters[currentWord][ii]+beta)/(topicCounters[ii] + nVocab*beta)
            probVector[ii] = probVector[ii]*(docTopicCounters[currentDoc][ii]+alpha)/(docCounters[currentDoc] + \
                                                                                        nTopics*alpha)
                                                                                                                       
    probVector = probVector/sum(probVector)
    probCumSum = np.cumsum(probVector)
    randInt = np.random.uniform(0,1,1)
    
    newTopic = sum(probCumSum < randInt)
    
    if newTopic != currentTopic:
        wordTopicCounters[currentWord][newTopic] += 1
        wordTopicCounters[currentWord][currentTopic] -= 1
        docTopicCounters[currentDoc][newTopic] += 1
        docTopicCounters[currentDoc][currentTopic] -= 1
        topicCounters[newTopic] += 1
        topicCounters[currentTopic] -= 1
    
    return newTopic, wordTopicCounters, docTopicCounters, topicCounters

In [286]:
def logGamma2(num):
    if num < 170:
        return np.log(gamma(num))
    else:
        num = num - 1
        return np.log(np.sqrt(2*np.pi*num)) + num*np.log(num/np.exp(1))

In [287]:
def getLogLik(allWords,topicLabels,nTopics,wordTopicCounters,topicCounters,beta):
    uniqueWords = np.unique(allWords)
    nVocab = len(uniqueWords)
    logLik = nTopics*(logGamma2(nVocab*beta)-nVocab*logGamma2(beta))
    for ii in xrange(0,nTopics):
        logLik = logLik - logGamma2(topicCounters[ii]+nVocab*beta)
        for jj in uniqueWords:
            logLik = logLik + logGamma2(wordTopicCounters[jj][ii]+beta)
    return logLik

In [298]:
def stationaryLDA(allWords,docLabel,nTopics):
  
    uniqueWords = np.unique(allWords)
    uniqueDocs = np.unique(docLabel)
    nVocab = len(uniqueWords)
    nDocs = len(uniqueDocs)
    
    theta = np.zeros((nDocs,nTopics))
    phi = np.zeros((nTopics,nVocab))
    
    # just need allWords, docLabel, topicLabel
    alpha = 0.001
    beta = 0.01
    
    nIterations = 300
    count = 0
    
    topicLabel, wordTopicCounters, docTopicCounters, topicCounters, docCounters = \
                                                                    drawTopicInit(allWords,docLabel,nTopics,alpha,beta)
                                                                                               
    logLik = np.zeros(nIterations+1)
    logLik[0] = getLogLik(allWords,topicLabel,nTopics,wordTopicCounters,topicCounters,beta)
    
    while count < nIterations:
        for ii in xrange(0,len(allWords)):
            topic, wordTopicCounters, docTopicCounters, topicCounters = drawTopic(ii,allWords,\
                              topicLabel,docLabel,nTopics,wordTopicCounters, docTopicCounters, \
                                                                                  topicCounters, docCounters,alpha,beta)
            topicLabel[ii] = topic
        # update phi and theta here, calculate P(w|z)
        for ii in xrange(0,nTopics):
            sumCurr = topicCounters[ii]
            for jj in xrange(0,nVocab):
                phi[ii,jj] = (wordTopicCounters[uniqueWords[jj]][ii] + beta)/(sumCurr + nVocab*beta)
        for ii in xrange(0,nDocs):
            sumCurr = docCounters[uniqueDocs[ii]]
            for jj in xrange(0,nTopics):
                theta[ii,jj] = (docTopicCounters[uniqueDocs[ii]][jj] + alpha)/(sumCurr + nTopics*alpha)
        logLik[count+1] = getLogLik(allWords,topicLabel,nTopics,wordTopicCounters,topicCounters,beta)
        count = count + 1

    return theta, phi, logLik, uniqueWords, uniqueDocs, topicLabel,\
                                            wordTopicCounters, topicCounters, docTopicCounters, docCounters

In [292]:
def findWord(wordArray,target):
    index = []
    for ii in xrange(0,len(wordArray)):
        if wordArray[ii] == target:
            index.append(ii)
    return index

In [293]:
def findWord2(words,target):
    index = []
    for ii in xrange(0,len(words)-len(target)):
        if words[ii:ii+len(target)] == target:
            index.append(ii)
    return index

In [299]:
textsTest0 = np.copy(textsTest)
allWords0,docLabel0,u0,n0 = preProcessWords(textsTest0)

In [300]:
startT = time.time()
nTopics = 10
theta,phi,logLikVec,uniqueWords,uniqueDocs,topicLabel,wordTopicCounters, topicCounters, docTopicCounters, docCounters = \
                    stationaryLDA(allWords0,docLabel0,nTopics)
endT = time.time()

In [301]:
print endT-startT

341.496000051


In [304]:
plt.plot(logLikVec)
plt.show()

# Print 10 topics here:

In [302]:
for ii in xrange(0,10):
    print uniqueWords[np.argsort(phi[ii,:])[-10:]]

['how' 'or' 'candidates' '#GOPDebate' 'many' 'I' 'he' '#CNNDeba' 'Trump'
 '#GOPDeba']
['I' 'looks' '#CNNDebate' '#GOPDebate' 'his' '#CNNDeba' 'Rand' 'Trump'
 'Paul' '#GOPDeba']
['SHOULDER' 'debate' 'TO' '#CNNDeba' 'Trump' 'he' '#GOPDebate' '2' 'time'
 '#GOPDeba']
['' 'A' 'an' 'Trump' 'he' 'debate' '#GOPDebate' 'I' '#CNNDeba' '#GOPDeba']
['he' 'trump' 'debate' 'Donald' '#GOPDebate' 'Trump' 'I' '#CNNDeba'
 '#GOPDeba' '']
['candidates' 'what' 'any' 'Trump' '#GOPDebate' '#CNNDebate' '' 'I'
 '#CNNDeba' '#GOPDeba']
['GOP' 'wearing' 'watch' '#CNNDebate' 'we' 'Trump' 'I' '#GOPDeba'
 '#CNNDeba' '#GOPDebate']
['debate' 'more' 'he' 'as' '#CNNDebate' 'Trump' '#GOPDebate' '#CNNDeba' 'I'
 '#GOPDeba']
['he' 'ere' "'m" '#CNNDebate' 'here' 'I' '#GOPDebate' 'we' '#CNNDeba'
 '#GOPDeba']
['arco' 'already' 'your' 'Trump' 'Cruz' 'joke' 'water' 'Rubio' '#CNNDeba'
 '#GOPDeba']
